In [3]:
! pip install dagshub

  Using cached anyio-4.5.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached graphql_core-3.2.5-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.3 MB 799.2 kB/s eta 0:00:16
   - -------------------------------------- 0.5/13.3 MB 799.2 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/13.3 MB 860.9 kB/s eta 0:00:15
   --- ------------------------------------ 1.0/13.3 MB 898.8 kB/s eta 0:00:14
   --- ------------------------------------ 1.3/13.3 MB 907.1 kB/s eta 0:00:14
   --- ------------------------------------ 1.3/13.3 MB 907.1 kB/s eta 0:00:14
   ---- ----------------------------------- 1.6/13.3 MB 922.4 kB/s eta 0:00:13
   ----- ---------------------------------- 1.8/13.3 MB 949.8 kB/s eta 0:00:13
   ------ ------------------

In [1]:
import os

In [2]:
os.chdir("../")

In [73]:
import dagshub
dagshub.init(repo_owner='vinu0404', repo_name='Mlops_Cancer_classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

[2024-12-28 20:21:54,300:INFO:_client:HTTP Request: GET https://dagshub.com/api/v1/repos/vinu0404/Mlops_Cancer_classification "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "vinu0404/Mlops_Cancer_classification"

[2024-12-28 20:21:54,318:INFO:helpers:Initialized MLflow to track repo "vinu0404/Mlops_Cancer_classification"]


Repository vinu0404/Mlops_Cancer_classification initialized!

[2024-12-28 20:21:54,325:INFO:helpers:Repository vinu0404/Mlops_Cancer_classification initialized!]


In [6]:
import tensorflow as tf


In [51]:
import mlflow

In [52]:
import mlflow.keras

In [7]:
model=tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [ ]:
from src.cnn.config.configuration import ConfigurationManager
from src.cnn.utils.common import read_yaml,make_dir
from src.cnn.constants.constants import *
#from src.cnn.constants.constants import MLFLOW_UI
from urllib.parse import urlparse


ImportError: cannot import name 'MLFLOW_UI' from 'src.cnn.constants.constants' (c:\Users\PC\Documents\mlops\src\cnn\constants\constants.py)

In [47]:
import os
from pathlib import Path
import yaml
import json
import joblib
import base64
from ensure import ensure_annotations
from box import ConfigBox
from box.exceptions import BoxValueError
from cnn import logger

def save_json(path:Path,data:dict):
    with open(path,"w") as f:
        json.dump(data,f,indent=4)
      #  logger.info(f"json file save at :{path}")

In [71]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        make_dir([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chest-CT-Scan-data")
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(training_data),
            mlflow_uri="https://dagshub.com/vinu0404/Mlops_Cancer_classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [49]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        print(f"{mlflow.set_registry_uri(self.config.mlflow_uri)}")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            print(f"{mlflow.log_params(self.config.all_params)}")
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="vin")
                print(f"vin completed")
            else:
                mlflow.keras.log_model(self.model, "model")

In [72]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-12-28 19:44:40,314:INFO:common:YAML file loaded successfully from config\config.yaml]
[2024-12-28 19:44:40,326:INFO:common:YAML file loaded successfully from params.yaml]
[2024-12-28 19:44:40,329:INFO:common:directory created at artifacts]


Found 102 images belonging to 2 classes.
7/7 [==============================] - 23s 3s/step - loss: 15.0486 - accuracy: 0.4314


2024/12/28 19:45:05 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-28 19:45:07,534:WARNING:save:Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\PC\AppData\Local\Temp\tmpr5ewtrvp\model\data\model\assets
[2024-12-28 19:45:08,729:INFO:builder_impl:Assets written to: C:\Users\PC\AppData\Local\Temp\tmpr5ewtrvp\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/12/28 19:46:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.


In [ ]:
from mlflow.tracking import MlflowClient

In [ ]:
client = MlflowClient()
client.transition_model_version_stage(
    name="VGG16Model", version=4, stage="Production"
)

RestException: RESOURCE_DOES_NOT_EXIST: Response: {'error_code': 'RESOURCE_DOES_NOT_EXIST'}

In [1]:
! pip install python-dotenv
